In [1]:
import os, time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
search_url = "https://www.google.com/search?sca_esv=554181109&rlz=1C1ONGR_enIN1041IN1053&sxsrf=AB5stBiFxmBbOYpInCdbKouAPdTBI_988w:1691302134273&q=cars&tbm=isch&source=lnms&sa=X&ved=2ahUKEwis4a63r8eAAxW1XGwGHSAXC98Q0pQJegQICxAB&biw=1366&bih=619&dpr=1"
max_links_to_fetch = 3
wd = webdriver.Chrome()
sleep_between_interactions = 1
wd.get(search_url)

In [3]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(sleep_between_interactions)

In [4]:
image_url = set()
image_count = 0
results_start = 0

In [5]:
while image_count < max_links_to_fetch:
    scroll_to_end(wd)
    thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")
    number_of_results = len(thumbnail_results)

    print(f"found : {thumbnail_results} search results. Extracting links from {results_start}:{number_of_results}")

    for img in thumbnail_results[results_start:number_of_results]:
        try:
            img.click()
            time.sleep(sleep_between_interactions)
        except Exception:
            continue

        actual_images = wd.find_elements(By.CSS_SELECTOR, "img.r48jcc")

        for actual_image in actual_images:
            if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                image_url.add(actual_image.get_attribute('src'))

        image_count = len(image_url)

        if len(image_url) >= max_links_to_fetch:
            print(f"found {len(image_url)} image links, done!")
            break

        else:
            print(f"found:", len(image_url), "image links, looking for more...")
            time.sleep(30)
        
        load_more_button = wd.find_elements(By.CSS_SELECTOR, ".LZ4I")

        if load_more_button:
            wd.execute_script("document.querySelector('.LZ4I').click();")

    results_start = len(thumbnail_results)

print(image_url)


found: 0 image links, looking for more...
found: 1 image links, looking for more...
found: 2 image links, looking for more...
found 3 image links, done!
{'https://ds-prod.citroen.in/static-assets/ds-static/s3fs-public/2023-04/1_mobile__.jpg?v5XyoHqW0vYdle1aYHEW6IvjQR89GoZl', 'https://www.autocar.co.uk/sites/autocar.co.uk/files/styles/body-image/public/99-best-luxury-cars-2023-bmw-i7-lead.jpg?itok=3aHfyLGd', 'https://carwow-uk-wp-3.imgix.net/18015-MC20BluInfinito-scaled-e1666008987698.jpg'}


In [8]:
number_of_results

48

In [26]:
def fetch_image_urls(query : str, max_links_to_fetch : int, wd : webdriver, sleep_between_interactions : int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # Build the google url
    search_url = "https://www.google.com/search?sca_esv=554181109&rlz=1C1ONGR_enIN1041IN1053&sxsrf=AB5stBiFxmBbOYpInCdbKouAPdTBI_988w:1691302134273&q={q}&tbm=isch&source=lnms&sa=X&ved=2ahUKEwis4a63r8eAAxW1XGwGHSAXC98Q0pQJegQICxAB&biw=1366&bih=619&dpr=1"

    # Load the page
    wd.get(search_url.format(q=query))

    image_url = set()
    image_count = 0
    results_start = 0

    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")
        number_of_results = len(thumbnail_results)

        print(f"found : {thumbnail_results} search results. Extracting links from {results_start}:{number_of_results}")

        for img in thumbnail_results[results_start:number_of_results]:
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            actual_images = wd.find_elements(By.CSS_SELECTOR, "img.r48jcc")

            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_url.add(actual_image.get_attribute('src'))

            image_count = len(image_url)

            if len(image_url) >= max_links_to_fetch:
                print(f"found {len(image_url)} image links, done!")
                break

            else:
                print(f"found:", len(image_url), "image links, looking for more...")
                time.sleep(30)
            
            load_more_button = wd.find_elements(By.CSS_SELECTOR, ".LZ4I")

            if load_more_button:
                wd.execute_script("document.querySelector('.LZ4I').click();")

        results_start = len(thumbnail_results)

    return image_url

In [27]:
def persist_image(folder_path : str, url : str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"Error - could not download {url} - {e}")
    
    try:
        f  = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), "wb")
        f.write(image_content)
        f.close()
        print(f"success - saved {url} - as {folder_path}")
    
    except Exception as e:
        print(f"Error - could not save {url} - {e}")

In [28]:
def search_and_download(search_term : str, number_of_images = 10):
    save_dir = "images/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    target_folder = os.path.join(save_dir, "_".join(search_term.lower().split(" ")))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome() as wd:
        res = fetch_image_urls(search_term, number_of_images, wd=wd, sleep_between_interactions=0.5)
    
    counter = 0
    for element in res:
        persist_image(target_folder, element, counter)
        counter = counter + 1

In [29]:
search_and_download(search_term="M S Dhoni")

found : [<selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba193f00", element="4F433AB85892BACD732B022AA4A1E857_element_17")>, <selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba193f00", element="4F433AB85892BACD732B022AA4A1E857_element_19")>, <selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba193f00", element="4F433AB85892BACD732B022AA4A1E857_element_21")>, <selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba193f00", element="4F433AB85892BACD732B022AA4A1E857_element_23")>, <selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba193f00", element="4F433AB85892BACD732B022AA4A1E857_element_25")>, <selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba193f00", element="4F433AB85892BACD732B022AA4A1E857_element_27")>, <selenium.webdriver.remote.webelement.WebElement (session="d7b930fcb01a86f53ea2e7ffba